In [ ]:
!pip install gdown
!gdown --id 1CowDEwmEOTZ38IhsEMfhxhPrWcbKZLwD
!gdown --id 1PInO2MlmZRa7Yp7fwBQsJBnWJFuJSZV1

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CowDEwmEOTZ38IhsEMfhxhPrWcbKZLwD
To: /content/truama.xlsx
100% 309k/309k [00:00<00:00, 5.96MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PInO2MlmZRa7Yp7fwBQsJBnWJFuJSZV1
To: /content/bloodGlucose.xlsx
100% 244k/244k [00:00<00:00, 5.62MB/s]


In [ ]:
import pandas as pd

# Load data
dataset1 = pd.read_excel("bloodGlucose.xlsx")

# Separate static and time-series features
static_features_db1 = dataset1[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI']]
time_series_features_db1 = dataset1[['weight0', 'glu41', 'weight1', 'glu4b', 'weight2', 'glu4c', 'weight3', 'glu4d', 'weight4']]
target_db1 = dataset1['干预']

dataset2 = pd.read_excel('truama.xlsx')

# Separate Features and Target for Dataset 2
static_features_db2 = dataset2[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层',
                                 '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重',
                                 '身高', '孕前BMI', '意外怀孕', '助孕治疗', '已生孩子数分层',
                                 'ACE1', 'ACE2', 'ACE3', 'ACE4', 'ACE5', 'ACE6', 'ACE7', 'ACE8',
                                 'ACE9', 'ACE10', 'F1a', 'F2a', 'F3a', 'F4a', 'F5a', 'F6a',
                                 'F7a', 'F8a', 'F9a', 'F10a', 'F11a', 'F12a', 'F13a', 'F14a',
                                 'F15a', 'F16a', 'F17a', 'F18a', 'F19a']]
target_db2 = dataset2['干预']

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standarization
scaler = StandardScaler()

static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler.fit_transform(static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']])
static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler.fit_transform(static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']])

<ipython-input-3-95f3ca595cf9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler.fit_transform(static_features_db1[['年龄', '身高', '孕前BMI', '孕前体重']])
<ipython-input-3-95f3ca595cf9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']] = scaler.fit_transform(static_features_db2[['年龄', '身高', '孕前BMI', '孕前体重']])


In [ ]:
pip install dtaidistance

Method 1

In [ ]:
# 用了BCEWithLogitsLoss(接受 logits 作为输入，而不是 Sigmoid 激活后的概率) + pos_weight(), ReduceLROnPlateau Scheduler
from sklearn.metrics import precision_recall_curve
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import recall_score, precision_score, fbeta_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torch.nn.functional import sigmoid
from torch.utils.data import Subset
def find_best_threshold(outputs, targets, beta=2, min_threshold=0.1):
    precision, recall, thresholds = precision_recall_curve(targets, outputs)

    # Compute F-beta score for thresholds
    fbeta_scores = (1 + beta**2) * (precision[:-1] * recall[:-1]) / ((beta**2 * precision[:-1]) + recall[:-1] + 1e-5)

    # Filter out thresholds that are too low
    valid_idx = thresholds >= min_threshold
    thresholds = thresholds[valid_idx]
    fbeta_scores = fbeta_scores[valid_idx]

    if len(fbeta_scores) == 0:
        # Fallback in case all thresholds are filtered out
        return min_threshold, 0.0

    best_idx = np.argmax(fbeta_scores)
    return thresholds[best_idx], fbeta_scores[best_idx]
# Shared features
shared_features_db2 = static_features_db2[['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层',
                                 '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重',
                                 '身高', '孕前BMI']].to_numpy()
# Dataset 2-specific features
specific_features_db2 = static_features_db2[['意外怀孕', '助孕治疗', '已生孩子数分层',
                                 'ACE1', 'ACE2', 'ACE3', 'ACE4', 'ACE5', 'ACE6', 'ACE7', 'ACE8',
                                 'ACE9', 'ACE10', 'F1a', 'F2a', 'F3a', 'F4a', 'F5a', 'F6a',
                                 'F7a', 'F8a', 'F9a', 'F10a', 'F11a', 'F12a', 'F13a', 'F14a',
                                 'F15a', 'F16a', 'F17a', 'F18a', 'F19a']].to_numpy()
shared_features_db1 = static_features_db1.to_numpy()
targets_db1 = target_db1.to_numpy()
targets_db2 = target_db2.to_numpy()
# 对共享特征进行SMOTE采样
smote = SMOTE(random_state=42)

# 对共享特征和目标进行重采样
shared_features_resampled_db1, targets_resampled_db1 = smote.fit_resample(shared_features_db1, targets_db1)
shared_features_resampled_db2, targets_resampled_db2 = smote.fit_resample(shared_features_db2, targets_db2)


# 获取重采样后的索引 (仅适用于带索引的输入数据)
# 假设 shared_features_db1 和 shared_features_db2 的最后一列是原始索引
resampled_indices_db1 = shared_features_resampled_db1[:, -1].astype(int)  # 从最后一列提取索引

resampled_indices_db2 = shared_features_resampled_db2[:, -1].astype(int)


# 对独特特征进行重采样
specific_features_resampled_db2, _ = smote.fit_resample(specific_features_db2, targets_db2)

# Masking
glu_features = time_series_features_db1[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).infer_objects(copy=False).to_numpy()
weight_features = time_series_features_db1[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).infer_objects(copy=False).to_numpy()
glu_features = np.nan_to_num(glu_features.astype(np.float32), nan=0.0)
weight_features = np.nan_to_num(weight_features.astype(np.float32), nan=0.0)
# Reshape to (samples, channels, time_steps)
glu_features = glu_features.reshape(glu_features.shape[0], 4, 1)  # (samples, time_steps, input_size)
weight_features = weight_features.reshape(weight_features.shape[0], 5, 1)


# 对时间序列特征进行对齐 (基于 resampled_indices_db1)
glu_features_resampled = glu_features[resampled_indices_db1]
weight_features_resampled = weight_features[resampled_indices_db1]

class CustomDataset(Dataset):
    def __init__(self, shared_features, glu_features=None, weight_features=None, unique_features=None, targets=None):
        """
        初始化数据集
        :param shared_features: 共享特征 (numpy array)
        :param glu_features: 时间序列特征 glu (numpy array, 可选)
        :param weight_features: 时间序列特征 weight (numpy array, 可选)
        :param unique_features: 独特特征 (numpy array, 可选)
        :param targets: 标签 (numpy array)
        """
        self.shared_features = torch.tensor(shared_features, dtype=torch.float32)
        self.glu_features = (
            torch.tensor(glu_features, dtype=torch.float32) if glu_features is not None else None
        )
        self.weight_features = (
            torch.tensor(weight_features, dtype=torch.float32) if weight_features is not None else None
        )
        self.unique_features = (
            torch.tensor(unique_features, dtype=torch.float32) if unique_features is not None else None
        )
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.shared_features)

    def __getitem__(self, idx):
        sample = {"shared_features": self.shared_features[idx]}
        if self.glu_features is not None:
            sample["glu_features"] = self.glu_features[idx]
        if self.weight_features is not None:
            sample["weight_features"] = self.weight_features[idx]
        if self.unique_features is not None:
            sample["unique_features"] = self.unique_features[idx]
        sample["target"] = self.targets[idx]
        return sample


db1_train_idx, db1_test_idx = train_test_split(
    np.arange(len(targets_resampled_db1)),
    test_size=0.1,
    stratify=targets_resampled_db1,
    random_state=42
)

db2_train_idx, db2_test_idx = train_test_split(
    np.arange(len(targets_resampled_db2)),
    test_size=0.1,
    stratify=targets_resampled_db2,
    random_state=42
)

# 使用 CustomDataset 实例创建完整数据集
db1_dataset = CustomDataset(
    shared_features=shared_features_resampled_db1,
    glu_features=glu_features_resampled,
    weight_features=weight_features_resampled,
    targets=targets_resampled_db1
)

db2_dataset = CustomDataset(
    shared_features=shared_features_resampled_db2,
    unique_features=specific_features_resampled_db2,
    targets=targets_resampled_db2
)

# 基于分层采样的索引创建训练集和测试集的 Subset
db1_train_dataset = Subset(db1_dataset, db1_train_idx)
db1_test_dataset = Subset(db1_dataset, db1_test_idx)

db2_train_dataset = Subset(db2_dataset, db2_train_idx)
db2_test_dataset = Subset(db2_dataset, db2_test_idx)

# 创建 DataLoader
db1_train_loader = DataLoader(db1_train_dataset, batch_size=32, shuffle=True)
db1_test_loader = DataLoader(db1_test_dataset, batch_size=32, shuffle=False)

db2_train_loader = DataLoader(db2_train_dataset, batch_size=32, shuffle=True)
db2_test_loader = DataLoader(db2_test_dataset, batch_size=32, shuffle=False)


class SharedFeatureModel(nn.Module):
    def __init__(self):
        super(SharedFeatureModel, self).__init__()
        self.shared_branch = nn.Sequential(
            nn.Linear(12, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        # LSTM branch for glu_features
        self.glu_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.glu_bn = nn.BatchNorm1d(64)  # Batch Normalization for LSTM outputs
        self.glu_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # LSTM branch for weight_features
        self.weight_lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.weight_bn = nn.BatchNorm1d(64)  # Batch Normalization for LSTM outputs
        self.weight_fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.db2_branch = nn.Sequential(
            nn.Linear(32, 64),  # 12 是 unique_features_db2 的输入维度
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),  # 输出维度为 32
            nn.BatchNorm1d(32),
            nn.ReLU()
        )


        # 推迟 fc 初始化
        self.fc = None



    def forward(self, shared_features, glu_features=None, weight_features=None, unique_features_db2=None, db="db1"):
        shared_output = self.shared_branch(shared_features)
        outputs = [shared_output]

        if glu_features is not None:
            glu_lstm_out, _ = self.glu_lstm(glu_features)
            glu_lstm_out = glu_lstm_out[:, -1, :]
            glu_bn_out = self.glu_bn(glu_lstm_out)
            glu_output = self.glu_fc(glu_bn_out)
            outputs.append(glu_output)

        if weight_features is not None:
            weight_lstm_out, _ = self.weight_lstm(weight_features)
            weight_lstm_out = weight_lstm_out[:, -1, :]
            weight_bn_out = self.weight_bn(weight_lstm_out)
            weight_output = self.weight_fc(weight_bn_out)
            outputs.append(weight_output)

        if db == "db2" and unique_features_db2 is not None:
            unique_output = self.db2_branch(unique_features_db2)
            outputs.append(unique_output)

        combined = torch.cat(outputs, dim=1)

        if self.fc is None or combined.shape[1] != self.fc[0].in_features:
            total_features = combined.shape[1]
            self.fc = nn.Sequential(
                nn.Linear(total_features, 64),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(64, 1)  # No Sigmoid here
            )


        output = self.fc(combined)
        return output


# Calculate pos_weight for db1
num_positive_db1 = targets_resampled_db1.sum()
num_negative_db1 = len(targets_resampled_db1) - num_positive_db1
pos_weight_db1 = torch.tensor([num_negative_db1 / num_positive_db1], dtype=torch.float32)

# Calculate pos_weight for db2
num_positive_db2 = targets_resampled_db2.sum()
num_negative_db2 = len(targets_resampled_db2) - num_positive_db2
pos_weight_db2 = torch.tensor([num_negative_db2 / num_positive_db2], dtype=torch.float32)


# 初始化模型
shared_feature_dim = shared_features_resampled_db1.shape[1]  # 共享特征维度
db2_unique_feature_dim = specific_features_resampled_db2.shape[1]  # db2独特特征维度


model = SharedFeatureModel()
# Use pos_weight for db1 and db2
criterion_db1 = nn.BCEWithLogitsLoss(pos_weight=pos_weight_db1)
criterion_db2 = nn.BCEWithLogitsLoss(pos_weight=pos_weight_db2)

optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
def train_model(model, db1_train_loader, db1_test_loader, db2_train_loader, db2_test_loader,
                            optimizer, criterion_db1, criterion_db2, epochs=100, beta=2, min_threshold=0, switch_interval=1):
    """
    交替训练模型并在每个 epoch 后评估
    :param model: 共享模型
    :param db1_train_loader: 数据集 db1 的训练数据加载器
    :param db1_test_loader: 数据集 db1 的测试数据加载器
    :param db2_train_loader: 数据集 db2 的训练数据加载器
    :param db2_test_loader: 数据集 db2 的测试数据加载器
    :param optimizer: 优化器
    :param criterion_db1: 数据集 db1 的损失函数
    :param criterion_db2: 数据集 db2 的损失函数
    :param epochs: 总训练轮数
    :param beta: F-beta 分数的 beta 值
    :param min_threshold: 最小阈值
    :param switch_interval: 多少个 epoch 切换一次数据集
    """
    for epoch in range(epochs):
        # 决定当前使用的数据集
        if (epoch // switch_interval) % 2 == 0:
            train_loader = db1_train_loader
            test_loader = db1_test_loader
            criterion = criterion_db1
            db = "db1"
        else:
            train_loader = db2_train_loader
            test_loader = db2_test_loader
            criterion = criterion_db2
            db = "db2"

        model.train()
        total_loss = 0.0
        for batch in train_loader:
            shared_features = batch["shared_features"]
            glu_features = batch.get("glu_features", None)
            weight_features = batch.get("weight_features", None)
            unique_features = batch.get("unique_features", None)
            targets = batch["target"].unsqueeze(1)  # (N,) -> (N, 1)

            optimizer.zero_grad()

            # 模型前向传播
            outputs = model(
                shared_features=shared_features,
                glu_features=glu_features,
                weight_features=weight_features,
                unique_features_db2=unique_features,
                db=db
            )

            # 计算损失
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            # 反向传播和优化
            loss.backward()
            optimizer.step()

        # 评估模型
        model.eval()
        all_targets = []
        all_outputs = []

        with torch.no_grad():
            for batch in test_loader:
                shared_features = batch["shared_features"]
                glu_features = batch.get("glu_features", None)
                weight_features = batch.get("weight_features", None)
                unique_features = batch.get("unique_features", None)
                targets = batch["target"].unsqueeze(1)  # (N,) -> (N, 1)

                # 模型前向传播
                outputs = model(
                    shared_features=shared_features,
                    glu_features=glu_features,
                    weight_features=weight_features,
                    unique_features_db2=unique_features,
                    db=db
                )

                # 收集预测值和真实标签
                all_targets.extend(targets.cpu().numpy())
                all_outputs.extend(outputs.cpu().numpy())

        # 计算评估指标
        all_targets = np.array(all_targets).flatten()
        all_outputs = np.array(all_outputs).flatten()

        best_threshold, best_fbeta = find_best_threshold(all_outputs, all_targets, beta=beta, min_threshold=min_threshold)

        predictions = (all_outputs >= best_threshold).astype(int)
        precision = precision_score(all_targets, predictions)
        recall = recall_score(all_targets, predictions)
        f2_score = fbeta_score(all_targets, predictions, beta=beta)
        accuracy = accuracy_score(all_targets, predictions)

        print(
            f"Epoch [{epoch+1}/{epochs}] ({db}), Loss: {total_loss/len(train_loader):.4f}, "
            f"Best Threshold: {best_threshold:.4f}, Recall: {recall:.4f}, "
            f"Precision: {precision:.4f}, F{beta} Score: {f2_score:.4f}, Accuracy: {accuracy:.4f}"
        )


print("Alternating training on db1 and db2...")
train_model(
    model,
    db1_train_loader, db1_test_loader,
    db2_train_loader, db2_test_loader,
    optimizer, criterion_db1, criterion_db2,
    epochs=100, switch_interval=1
)



<ipython-input-5-b1a1d0704268>:65: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  glu_features = time_series_features_db1[['glu41', 'glu4b', 'glu4c', 'glu4d']].replace([' ', ''], np.nan).infer_objects(copy=False).to_numpy()
<ipython-input-5-b1a1d0704268>:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weight_features = time_series_features_db1[['weight0', 'weight1', 'weight2', 'weight3', 'weight4']].replace([' ', ''], np.nan).infer_objects(copy=False).to_numpy()
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:

Alternating training on db1 and db2...
Epoch [1/100] (db1), Loss: 0.6934, Best Threshold: 0.0233, Recall: 0.0341, Precision: 0.7500, F2 Score: 0.0421, Accuracy: 0.5114
Epoch [2/100] (db2), Loss: 0.6968, Best Threshold: 0.0085, Recall: 0.0349, Precision: 0.6000, F2 Score: 0.0430, Accuracy: 0.5058
Epoch [3/100] (db1), Loss: 0.6885, Best Threshold: 0.0017, Recall: 0.2159, Precision: 0.6552, F2 Score: 0.2493, Accuracy: 0.5511
Epoch [4/100] (db2), Loss: 0.6928, Best Threshold: 0.0029, Recall: 0.8256, Precision: 0.5338, F2 Score: 0.7442, Accuracy: 0.5523
Epoch [5/100] (db1), Loss: 0.6845, Best Threshold: 0.0086, Recall: 0.2727, Precision: 0.6154, F2 Score: 0.3069, Accuracy: 0.5511
Epoch [6/100] (db2), Loss: 0.7032, Best Threshold: 0.0024, Recall: 0.8023, Precision: 0.5188, F2 Score: 0.7233, Accuracy: 0.5291
Epoch [7/100] (db1), Loss: 0.6867, Best Threshold: 0.0016, Recall: 0.4886, Precision: 0.7049, F2 Score: 0.5206, Accuracy: 0.6420
Epoch [8/100] (db2), Loss: 0.6975, Best Threshold: 0.0015,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [24/100] (db2), Loss: 0.7171, Best Threshold: 0.0000, Recall: 0.0000, Precision: 0.0000, F2 Score: 0.0000, Accuracy: 0.5000
Epoch [25/100] (db1), Loss: 0.6906, Best Threshold: 0.0007, Recall: 0.7500, Precision: 0.5366, F2 Score: 0.6947, Accuracy: 0.5511
Epoch [26/100] (db2), Loss: 0.7035, Best Threshold: 0.0016, Recall: 0.2791, Precision: 0.4800, F2 Score: 0.3046, Accuracy: 0.4884
Epoch [27/100] (db1), Loss: 0.6945, Best Threshold: 0.0068, Recall: 0.2386, Precision: 0.7000, F2 Score: 0.2749, Accuracy: 0.5682
Epoch [28/100] (db2), Loss: 0.6797, Best Threshold: 0.0110, Recall: 0.8605, Precision: 0.5692, F2 Score: 0.7806, Accuracy: 0.6047
Epoch [29/100] (db1), Loss: 0.6976, Best Threshold: 0.0379, Recall: 0.0682, Precision: 0.6667, F2 Score: 0.0831, Accuracy: 0.5170
Epoch [30/100] (db2), Loss: 0.7191, Best Threshold: 0.0094, Recall: 0.9651, Precision: 0.4940, F2 Score: 0.8105, Accuracy: 0.4884
Epoch [31/100] (db1), Loss: 0.6937, Best Threshold: 0.0355, Recall: 0.1932, Precision: 0.7

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [53/100] (db1), Loss: 0.6969, Best Threshold: 0.0000, Recall: 0.0000, Precision: 0.0000, F2 Score: 0.0000, Accuracy: 0.5000
Epoch [54/100] (db2), Loss: 0.6926, Best Threshold: 0.1095, Recall: 0.9419, Precision: 0.5827, F2 Score: 0.8385, Accuracy: 0.6337
Epoch [55/100] (db1), Loss: 0.6940, Best Threshold: 0.0165, Recall: 0.2614, Precision: 0.6970, F2 Score: 0.2987, Accuracy: 0.5739
Epoch [56/100] (db2), Loss: 0.6911, Best Threshold: 0.0117, Recall: 0.5116, Precision: 0.5641, F2 Score: 0.5213, Accuracy: 0.5581
Epoch [57/100] (db1), Loss: 0.6939, Best Threshold: 0.0023, Recall: 0.4545, Precision: 0.5634, F2 Score: 0.4728, Accuracy: 0.5511
Epoch [58/100] (db2), Loss: 0.6882, Best Threshold: 0.0010, Recall: 0.2791, Precision: 0.6486, F2 Score: 0.3150, Accuracy: 0.5640
Epoch [59/100] (db1), Loss: 0.6830, Best Threshold: 0.0042, Recall: 0.4432, Precision: 0.6724, F2 Score: 0.4756, Accuracy: 0.6136
Epoch [60/100] (db2), Loss: 0.6918, Best Threshold: 0.0059, Recall: 0.2558, Precision: 0.7

Code for creating the new test set.

In [ ]:
import pandas as pd
import numpy as np

# 提取重叠特征
shared_features = ['年龄', '居住地类别', '婚姻状况', '年收入分层', '学历分层', '工作状态', '吸烟分层', '被动吸烟分层', '饮酒分层', '孕前体重', '身高', '孕前BMI']

# 重置索引
db1_shared = dataset1[shared_features]
db2_shared = dataset2[shared_features]
db1_targets = dataset1['干预']
db2_targets = dataset2['干预']

db1_shared = db1_shared.reset_index(drop=True)
db2_shared = db2_shared.reset_index(drop=True)
db1_targets = db1_targets.reset_index(drop=True)
db2_targets = db2_targets.reset_index(drop=True)

# 提取独有特征（包括时间序列特征）
db1_unique_features = dataset1.drop(columns=shared_features + ['干预'])
db2_unique_features = dataset2.drop(columns=shared_features + ['干预'])

# 设置共享特征的相似阈值
similarity_threshold = 1e-3

# 创建新测试集
new_test_data = []

for i, row_db1 in db1_shared.iterrows():
    for j, row_db2 in db2_shared.iterrows():
        # 判断共享特征的相似性
        if np.allclose(row_db1.values, row_db2.values, atol=similarity_threshold):
            if db1_targets.iloc[i] == db2_targets.iloc[j]:
                # 默认重叠部分用 db2 的值
                combined_row = row_db2.to_dict()  # 转为字典，避免索引冲突

                # 补充 db1 和 db2 的独有特征
                db1_non_shared = db1_unique_features.iloc[i].to_dict()
                db2_non_shared = db2_unique_features.iloc[j].to_dict()

                # 将所有特征合并
                combined_row.update(db1_non_shared)  # 添加 db1 独有特征
                combined_row.update(db2_non_shared)  # 添加 db2 独有特征

                # 添加目标值
                combined_row['target'] = db1_targets.iloc[i]  # 使用相同的目标值

                # 追加到新测试集
                new_test_data.append(combined_row)

                # 匹配成功后立即跳出内循环
                break

# 将新测试集转换为 DataFrame
new_test_df = pd.DataFrame(new_test_data)

# 保存到 Excel 文件
output_file_path = "new_test_dataset_with_all_features.xlsx"
new_test_df.to_excel(output_file_path, index=False)

# 提供下载
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>